In [1]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd

In [2]:
def run_10yr_impact(reform, csv_path, label=""):
    """Run 10-year income tax impact analysis for a reform."""
    pd.DataFrame(columns=["year", "revenue_billions"]).to_csv(csv_path, index=False)
    
    total = 0
    for year in range(2026, 2036):
        baseline = Microsimulation()
        reformed = Microsimulation(reform=reform)
        
        baseline_tax = baseline.calculate("income_tax", period=year).sum() / 1e9
        reformed_tax = reformed.calculate("income_tax", period=year).sum() / 1e9
        
        revenue = reformed_tax - baseline_tax
        total += revenue
        print(f"{year}: ${revenue:.2f}B")
        
        pd.DataFrame([{"year": year, "revenue_billions": round(revenue, 2)}]).to_csv(
            csv_path, mode="a", header=False, index=False
        )
    
    pd.DataFrame([{"year": "Total", "revenue_billions": round(total, 2)}]).to_csv(
        csv_path, mode="a", header=False, index=False
    )
    print(f"\n10-Year Total{' (' + label + ')' if label else ''}: ${total:.2f}B")
    return total

In [ ]:
# Basic AGI surtax (1% on $100k/$200k, 2% on $1M)
reform_basic = Reform.from_dict({
    "gov.contrib.crfb.surtax.in_effect": {
        "2026-01-01.2100-12-31": True
    },
}, country_id="us")

run_10yr_impact(reform_basic, "agi_surtax_10yr_impact.csv", "Basic")

In [3]:
# Surtax with EXPANDED BASE (adds retirement contributions, HSA, tax-exempt income, etc.)
reform_expanded = Reform.from_dict({
    "gov.contrib.crfb.surtax.in_effect": {
        "2026-01-01.2100-12-31": True
    },
    "gov.contrib.crfb.surtax.increased_base.in_effect": {
        "2026-01-01.2100-12-31": True
    },
}, country_id="us")

run_10yr_impact(reform_expanded, "agi_surtax_expanded_base_10yr_impact.csv", "Expanded Base")

2026: $76.81B
2027: $81.25B
2028: $85.67B
2029: $91.08B
2030: $97.18B
2031: $103.80B
2032: $110.89B
2033: $118.33B
2034: $126.65B
2035: $135.49B

10-Year Total (Expanded Base): $1027.15B


np.float64(1027.1456010329425)

In [ ]:
# CBO Option 46b comparison: 2% flat surtax on AGI above $100k/$200k
reform_cbo_2pct = Reform.from_dict({
    "gov.contrib.crfb.surtax.in_effect": {
        "2026-01-01.2100-12-31": True
    },
    # Override to flat 2% (no tiered structure)
    "gov.contrib.crfb.surtax.rate.single[1].rate": {
        "2026-01-01.2100-12-31": 0.02
    },
    "gov.contrib.crfb.surtax.rate.single[2].rate": {
        "2026-01-01.2100-12-31": 0.02
    },
    "gov.contrib.crfb.surtax.rate.joint[1].rate": {
        "2026-01-01.2100-12-31": 0.02
    },
    "gov.contrib.crfb.surtax.rate.joint[2].rate": {
        "2026-01-01.2100-12-31": 0.02
    },
}, country_id="us")

run_10yr_impact(reform_cbo_2pct, "cbo_comparison_2pct_10yr.csv", "CBO 2% Flat")

In [5]:
baseline = Microsimulation()

In [8]:
baseline.calculate("traditional_403b_contributions", 2026).sum()

np.float64(0.0)